In [1]:
import pandas as pd
import numpy as np
from operator import attrgetter
import seaborn as sns
import datetime
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.cluster import KMeans
from scipy.stats import boxcox 
import requests
from urllib.parse import urlencode

 

In [ ]:
# загружаем данные (без скачивания файлов)

base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key = ['https://disk.yandex.ru/d/FUi5uSd6BfG_ig', 
              'https://disk.yandex.ru/d/t9Li4JOfgxuUrg',
              'https://disk.yandex.ru/d/Gbt-yAcQrOe3Pw']                # ссылки на данные

file_names = ['olist_customers_dataset.csv', 
             'olist_orders_dataset.csv',
             'olist_order_items_dataset.csv']                           # имена файлов

datasets = ['customers', 'orders', 'order_items']

for i in range(3):
    # получаем загрузочную ссылку
    final_url = base_url + urlencode(dict(public_key=public_key[i]))
    response = requests.get(final_url)
    download_url = response.json()['href']
    
    # загружаем данные по ссылкам
    globals()[datasets[i]] = pd.read_csv(download_url)

In [ ]:
print('Число строк в customers:  ', customers.shape[0])
print('Число строк в orders:     ', orders.shape[0])
print('Число строк в order_items:', order_items.shape[0])

In [ ]:
customers.head()

In [ ]:
customers.dtypes

In [ ]:
orders.head()

In [ ]:
orders.dtypes

In [ ]:
order_items.dtypes

In [ ]:
# список колонок со временем в orders
time_col_ls = ['order_purchase_timestamp', 
          'order_approved_at', 
          'order_delivered_carrier_date', 
          'order_delivered_customer_date', 
          'order_estimated_delivery_date']

In [ ]:
# переводим временные колонки orders в тип дат
orders[time_col_ls] = orders[time_col_ls].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d'))

# переводим временную колонку order_items в тип дат
order_items['shipping_limit_date'] = pd.to_datetime(order_items['shipping_limit_date'], format='%Y-%m-%d')
orders.dtypes

In [ ]:
orders.head()

In [ ]:
order_items.head()

In [ ]:
order_items.dtypes

In [ ]:
Что считать покупкой?
Первое, что приходит в голову - посмотреть order_approved_at (время подтверждения оплаты заказа). Оплатил - значит купил.
Проверим сколько записей в этой колонке отсутствует.



In [ ]:
orders.order_approved_at.isna().sum()

In [ ]:
# отберём только те, где order_approved_at с пропусками и посмотрим на значения order_status
(
    orders[orders.order_approved_at.isna()]
        .order_status
        .value_counts()
)

In [ ]:
'canceled' - здесь всё понятно, заказы с этим статусом нельзя считать покупками.
'created' - созданные, но, по всей видимости, ещё не оплаченные, а значит они ещё не куплены (и будут ли оплачены неизвестно).
'delivered' - доставленные пользователю, мне понимается это, как факт получения товара покупателем. Если нет даты подтверждения оплаты, возможно, имеет место какой-то сбой в системе.
Взглянем на детали заказа доставленных, но с пропусками даты оплаты.

Найдём список order_id из orders со статусом delivered и отфильтруем по нему order_items.



In [ ]:
# отфильтруем orders по пропускам в order_approved_at и order_status == "delivered"
delivered_not_approved = (
    orders
        .query('order_approved_at.isna() and order_status == "delivered"')
)
delivered_not_approved


In [ ]:
# отфильтруем order_items по значениям order_id из предыдущего шага
delivered_not_approved_items = (
    order_items
        .query('order_id in @delivered_not_approved.order_id')
)
delivered_not_approved_items


In [ ]:
# сравним количество строк отфильтрованных датафрэймов
print('Число строк в delivered_not_approved:      ', delivered_not_approved.shape[0])
print('Число строк в delivered_not_approved_items:', delivered_not_approved_items.shape[0])

In [ ]:
# найдём повторы order_id
order_id_duplicated = (
    delivered_not_approved_items
        .loc[
            delivered_not_approved_items
                .duplicated(subset='order_id', keep=False)
        ]
)
order_id_duplicated

In [ ]:
Совпадает всё, кроме order_item_id (идентификатор товара внутри одного заказа). Видимо это 2 одинаковых товара в одном заказе.
Попутно больше узнали о датасете olist_order_items_dataset.csv - скорее всего в нём содержатся записи по каждой единице товара разными строками, одинаковые в том числе.

Выясним, есть ли пропуски в order_status датафрэйма orders

In [ ]:
(
    orders
        .order_status
        .isna()
        .sum()
)

In [ ]:
Какой статус имеют заказы c подтверждением оплаты?

In [ ]:
# отберём только те, где order_approved_at без пропусков и посмотрим на значения order_status
(
    orders[orders.order_approved_at.isna() == False]
        .order_status
        .value_counts()
)

In [ ]:
Получается все, кроме 'created'.

'approved' — подвтверждён и оплачен — покупка;
'invoiced' — выставлен счёт и оплачен — покупка;
'processing' — оплачен и собирается — покупка;
'shipped' — оплачен и отгружен — покупка;
'delivered' — оплачен и доставлен - покупка;
'unavailable' — недоступен — скорее всего выполнить заказ не удастся;
'canceled' — отменён — нельзя считать покупкой.

В итоге, покупкой будем считать все статусы, кроме:

'created' (встречается только в заказах с пропусками оплаты),
'unavailable' (скорее всего выполнить не удастся),
'canceled' (отменён).
Значения в колонке order_approved_at учитывать не будем (учтено при выборе допустимых статусов).



In [ ]:
# список статусов, которые не будем считать покупками
no_purchases_statuses = ['created', 'unavailable', 'canceled']

In [ ]:
1. Сколько у нас пользователей, которые совершили покупку только один раз?
Теперь взглянем на датафрэйм customers. Хочется понять связь между customer_id (позаказный идентификатор пользователя) и customer_unique_id (уникальный идентификатор пользователя), чтобы решить по какой колонке смотреть покупки пользователей.
Сравним количество уникальных customer_id и order_id в customers и orders:

In [ ]:
print('orders: количество уникальных значений customer_id:   ', orders.customer_id.nunique())
print('orders: количество уникальных значений order_id:      ', orders.order_id.nunique())
print('orders: общее количество строк:                       ', orders.shape[0])
print('customers: количество уникальных значений customer_id:', customers.customer_id.nunique())
print('customers: общее количество строк:                    ', customers.shape[0])
print('customers: количество уникальных customer_unique_id:  ', customers.customer_unique_id.nunique())
print('customers: количество пропусков  customer_unique_id:  ', customers.customer_unique_id.isna().sum())

In [ ]:
Все customer_id в customers и orders уникальны, и им соответствуют уникальные order_id. По customer_id можно объединить эти датафрэймы. Если количество строк inner join при этом сохранится, значит значения customer_id в этих датафрэймах полностью совпадают.

Похоже на то, что на часть customer_unique_id приходится по несколько customer_id. Убедимся в этом визуально, показав дубликаты customer_unique_id.

In [ ]:
(
    customers
        .loc[customers
                .duplicated(keep=False, subset='customer_unique_id')]
        .sort_values('customer_unique_id')
        .head()
)

In [ ]:
Следовательно при оценке покупок необходимо ориентироваться на customer_unique_id. Для этого объединим датасеты customers и orders по колонке customer_id.

In [ ]:
customers_orders_merged = customers.merge(orders, on='customer_id')
customers_orders_merged.shape


In [ ]:
Количество строк у объединённого датафрэйма осталось таким же - значит все значения customer_id в обоих датафрэймах полностью совпадают и никакие значения при объединении не потеряны.

In [ ]:
customers_orders_merged.head()

In [ ]:
Число пользователей, совершивших покупку только один раз:


In [ ]:
(
    customers_orders_merged
        .query('order_status not in @no_purchases_statuses')  # убираем строки со статусами-непокупками
        .groupby('customer_unique_id', as_index=False)        # группируем по customer_unique_id
        .agg({'customer_id': 'count'})                        # подсчитываем количество покупок на каждого уник. пользователя
        .query('customer_id == 1')                            # отбираем тех, у кого количество покупок = 1
        .shape[0]
)

In [ ]:
Какой заказ считать недоставленным?
Здесь напрашиваются два варианта:

Всё, что имеет статус отличный от delivered;
Отсутствует дата доставки.
Проверим как согласуются колонки со статусом и временем доставки.

In [ ]:
# уникальные значения статуса среди непропущенных 
# значений order_delivered_customer_date в orders
(
    orders
        .query('order_delivered_customer_date.isna() == False')
        .order_status
        .value_counts()
)

In [ ]:
# отменённые заказы с датой доставки клиенту
(
    orders
        .query('order_delivered_customer_date.isna() == False and order_status == "canceled"')
)

In [ ]:
# значения order_status при пропущенных датах доставки в orders
(
    orders
        .query('order_delivered_customer_date.isna() == True')
        .order_status
        .value_counts()
)

In [ ]:
# orders с пропущенными датами достаки, при этом имеющие статус "доставлен"
(
    orders
        .query('order_delivered_customer_date.isna() == True and order_status == "delivered"')
)

In [ ]:
Здесь не наблюдается каких-то закономерностей, в силу неизвестных нам причин заказы одновременно имеют статус delivered и пропущенные значения order_delivered_customer_date.
Остановимся на версии №1: всё, что имеет статус отличный от delivered будем считать недоставленным.

2. Сколько заказов в месяц в среднем не доставляется по разным причинам?

In [ ]:
# датафрэйм с не доставленными заказами
not_delivered_orders = (
    orders
        .copy()
        .query('order_status != "delivered"')
)

In [ ]:
# статусы недоставленных и их количество
(
    not_delivered_orders
        .order_status
        .value_counts()
)

In [ ]:
# добавим колонку содержащую только год и месяц на основе order_estimated_delivery_date
not_delivered_orders['year_month_estimated'] = (
    not_delivered_orders
        .order_estimated_delivery_date
        .dt
        .strftime("%Y-%m")
)

In [ ]:
# сгруппируем по колонке "year_month_estimated" и статусу заказа, чтобы посчитать количество недоставленных
undelivered_by_month = (
    not_delivered_orders
        .groupby(['year_month_estimated', 'order_status'], as_index=False)
        .agg({'order_id': 'count'})
        .rename(columns={'order_id': 'undelivered'})
)
undelivered_by_month.head()

In [ ]:
# среднее количество недоставленных заказов в месяц
undelivered_by_month.undelivered.mean()


In [ ]:
Посчитаем среднее количество недоставленных заказов в месяц по каждму статусу

In [ ]:
# среднее количество недоставленных заказов по месяцам
mean_by_month_df = pd.DataFrame()

for status in undelivered_by_month \
                                .order_status \
                                .unique():
    temp_df = pd.DataFrame(                                                          
        [[status, 
          undelivered_by_month
                  .query('order_status == @status')
                  .undelivered
                  .mean()]], 
                columns=['order_status', 'mean_by_month']
                          )
    mean_by_month_df = pd.concat([mean_by_month_df, temp_df])

mean_by_month_df.sort_values('mean_by_month', inplace=True, ascending=False)
mean_by_month_df = mean_by_month_df.reset_index(drop=True)

In [ ]:
#Среднее количество недоставленных заказов в месяц по каждому статусу:
mean_by_month_df

In [ ]:
# недоставленные со статусом shipped
shipped = (
    not_delivered_orders
        .copy()
        .query('order_status == "shipped"')
)

In [ ]:
# разница во времени обещаной датой доставки и датой передачи в логистическую службу
shipped['delta_time'] = (
    shipped
        .order_estimated_delivery_date
            .sub(shipped
                 .order_delivered_carrier_date)
)

In [ ]:
# недоставленные shipped с отрицательной разницей во времени
shipped.query('delta_time < @pd.to_timedelta(0)')

In [ ]:
3. По каждому товару определить, в какой день недели товар чаще всего покупается.
Подробнее взглянем на order_items

Сравнение датафрэймов orders и order_items
Посмотрим на количество уникальных order_id в orders и order_items:

In [ ]:
print('orders: количество уникальных значений order_id:       ', orders.order_id.nunique())
print('orders: общее количество строк:                        ', orders.shape[0])
print('order_items: количество уникальных значений order_id:  ', order_items.order_id.nunique())
print('order_items: общее количество строк:                  ', order_items.shape[0])

In [ ]:
Общее количество строк order_items больше, чем в orders (как мы уже выяснили, на каждый order_id может приходиться несколько записей товаров), а уникальных значений order_id - меньше.
Взлянем на order_id, отсутствующие в order_items, но присутствующие в orders:



In [ ]:
# список order_id, которых нет в order_items
droped_order_id_ls = orders.query('order_id not in @order_items.order_id.unique()').order_id

# строки с order_id по которым нет данных в order_items
droped_order_id = orders.query('order_id in @droped_order_id_ls')

In [ ]:
# количество пропущеных order_id
droped_order_id.shape[0]

In [ ]:
droped_order_id.head()

In [ ]:
# Попытаемся найти закономерности

# уникальные значения order_status в строках с пропущенными order_id и их количество
droped_order_id.order_status.value_counts()

In [ ]:
# общее количетсво строк с order_status == "unavailable" в orders
orders.query('order_status == "unavailable"').shape[0]

In [ ]:
# общее количетсво строк с order_status == "canceled" в orders
orders.query('order_status == "canceled"').shape[0]


In [ ]:
# строки droped_order_id с order_status "created", "invoiced" или "shipped"
(
    droped_order_id
        .query('order_status in ["created", "invoiced", "shipped"]')
)

In [ ]:
# пропуски в колонках со временем droped_order_id
droped_order_id[time_col_ls].isna().sum()

In [ ]:
Все значения order_delivered_customer_date (время доставки заказа) пропущены и во всех значениях order_status нет:

delivered (доставлен пользователю),
approved (подтверждён),
processing (в процессе сборки заказа)
Большинство из них либо unavailable, либо canceled.

In [ ]:
# всего пропущено order_delivered_customer_date в orders
orders.order_delivered_customer_date.isna().sum()

In [ ]:
# всего пропусков order_estimated_delivery_date в orders
orders.order_estimated_delivery_date.isna().sum()

In [ ]:
# всего пропусков order_purchase_timestamp в orders
orders.order_purchase_timestamp.isna().sum()

In [ ]:
print('Самая ранняя дата order_estimated_delivery_date в orders:          ', orders.order_estimated_delivery_date.min())
print('Самая ранняя дата order_estimated_delivery_date в droped_order_id: ', droped_order_id.order_estimated_delivery_date.min())
print('Самая поздняя дата order_estimated_delivery_date в orders:         ',orders.order_estimated_delivery_date.max())
print('Самая поздняя дата order_estimated_delivery_date в droped_order_id:', droped_order_id.order_estimated_delivery_date.max())

In [ ]:
# order_id некупленных товаров
no_purchases_order_id = (
    orders
        .query('order_status == @no_purchases_statuses')
    .order_id
)

# количетсво строк с заказами в order_items, которые мы не считаем покупками
(
    order_items
        .query('order_id in @no_purchases_order_id')
        .shape[0]
)

In [ ]:
Итак, всё, что можно сказать о отсутствующих строках в order_items - это часть недоставленных товаров, большая из которых - недоступна или отменена. Почти все недоступные не попали в order_items..
Также в в order_items имеются заказы, которые мы не считаем покупками.

Объединим все датафрэймы в один

In [ ]:
total_df = customers_orders_merged.merge(order_items, on='order_id')

In [ ]:
total_df.shape

In [ ]:
order_items.shape


In [ ]:
total_df.head()

In [ ]:
#Отберём те строки, что считаются покупками

purchase_total_df = (
    total_df
        .copy()
        .query('order_status not in @no_purchases_statuses')
)
purchase_total_df.head()

In [ ]:
# количество строк с пропущенными датами оплаты в purchase_total_df
purchase_total_df.order_approved_at.isna().sum()

In [ ]:
#Добавим колонки дней недели на основе order_purchase_timestamp.

# добавляем колонку с днями недели создания заказа
purchase_total_df['order_weekday'] = (
    purchase_total_df
        .copy()
        .order_purchase_timestamp
        .dt
        .day_name()
)

In [ ]:
purchase_total_df.head()

In [ ]:
За одну покупку определённого товара будем считать его наличие в заказе не зависимо от количества. Т.е. если в заказе, например, 5 одинаковых товаров - это одна покупка, а не 5.

Одинаково часто встречающихся дней недели может быть несколько, чтобы это учесть создадим функцию, отбирающую все дни недели, которые встречаются максимально часто и преобразует их в строку.

In [ ]:
# функция собирающая максимально часто встречающиеся дни недели в одну строку
def get_weekdays(weekdays):
    max_vals = weekdays.value_counts().max()
    weekday_list = (weekdays 
            .value_counts()[weekdays.value_counts() == max_vals] 
            .index 
            .tolist())
    return ', '.join(weekday_list)

In [ ]:
#Дни недели, в которые чаще всего покупаются товары.
%%time
purchase_weekday_by_product = (
    purchase_total_df
        .groupby(['order_id', 'product_id'], as_index=False)  # сначала избавимся от повторов product_id в заказе: группируем по двум колонкам
        .agg({'order_weekday': 'first'})                      # выбираем первое значение, т.к. они все одинаковые
        .groupby('product_id', as_index=False)                # затем снова группируем, чтобы схлопнуть в одно значение (оставить уникальные)
        .agg({'order_weekday': lambda x: get_weekdays(x)})    # подсчитываем и выбираем значение с максимальным "счётом"
)


In [ ]:
purchase_weekday_by_product.head()

In [ ]:
#4. Сколько у каждого из пользователей в среднем покупок в неделю (по месяцам)?
Внутри месяца может быть не целое количество недель. Например, в ноябре 2021 года 4,28 недели. И внутри метрики это нужно учесть.



In [ ]:
purchase_total_df.head()

In [ ]:
#Расчитаем и добавим колонку mean_orders_per_week со значениями средним значением покупок в неделю по месяцам для каждого пользователя.

mean_per_week_df = (
    purchase_total_df
        .groupby(['year_month_order', 'customer_unique_id'], as_index=False)             # группируем по году-месяцу и пользователю
        .agg({'order_purchase_timestamp': 'count'})                                      # считаем количество (в месяц на пользователя)
        .assign(weeks_in_month = lambda x: x.year_month_order.dt.daysinmonth / 7)        # добавляем колонку количество недель в текущем месяце
        .rename(columns={'order_purchase_timestamp': 'orders_per_month'})                # переименовываем колонку с подсчётом
        .assign(mean_orders_per_week = lambda x: x.orders_per_month / x.weeks_in_month)  # добавляем колонку в которой делим количество заказов в месяц на количество недель в месяце
        [['year_month_order', 'customer_unique_id', 'mean_orders_per_week']]             # отберём нужные колонки
)

In [ ]:
#Итак, количество покупок каждого пользоватлея в неделю по месяцам
mean_per_week_df.head()


In [ ]:
5. Написать функцию на python, позволяющую строить когортный анализ. В период с января по декабрь выявить когорту с самым высоким retention на 3-й месяц.

In [ ]:
# количество уникальных пользователей
num_customers = (
    purchase_total_df
        .groupby(['customer_unique_id'])
        .order_id
        .nunique()
)
# доля пользователей сделавших более 1 заказа
mult_orders_perc = (
    np.sum(num_customers > 1) / purchase_total_df
        .order_id.nunique()
)
print(f'{100 * mult_orders_perc:.2f}% пользователей сделали более одного заказа.')

In [ ]:
# функция для когортного анализа
def user_retention(data, customer_id, date, period='M'):
    '''
    Функция строит retention-матрицу когортного анализа (возвращает датафрэйм)
    Требуются библиотеки: 
        from operator import attrgetter
        import pandas as pd
    Описание полей:
    data        - датафрэйм
    customer_id - колонка с id клиентов (str)
    order_date  - колонка с датами (datetime)
    period      - период разбивки (str, по умолчанию "M" - месяц). Все псевдонимы: https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#timeseries-offset-aliases
    '''
    data = data.copy()
    data[date] = data[date].dt.to_period('M')      # переводим колонку дат в тип "период"
    
    # создаём когорты
    users_cohorts = (
        data
            .groupby(customer_id, as_index=False)  # группируем по пользователям
            [date]                                 # выбираем колонку с датами
            .min()                                 # отбираем минимальные колонки для каждого пользователя
            .rename(columns={date: 'cohort'})      # переименовываем колонку с дамами в когорты
    )
    
    # объединим датасеты
    cohorts_df = (
        users_cohorts                                           # датасет с когортами
            .merge(data[[customer_id, date]], on=customer_id)   # объединяем с основным датасетом по пользователям
            .sort_values('cohort')                              # сортируем по когортам
            .rename(columns={date: 'target_action_dates'})      # переименовываем колонку с датами целевых действий
    )
    
    # сгруппируем по когортам и месяцам покупок и посчитаем количество покупателей на каждый месяц
    cohorts_df = (
        cohorts_df
            .groupby(['cohort', 'target_action_dates'], as_index=False)
            .agg(customer_quantity = (customer_id, 'nunique'))
    )
    
    # считаем разницу между первой и последующими покупками, добавляем в столбец
    cohorts_df['period_number'] = (
        cohorts_df
            .target_action_dates                  
            .sub(cohorts_df.cohort)               # вычитаем когорты из времени целевых действий
            .apply(attrgetter('n'))               # передаём атрибут номинальной единицы (т.е. будут те же что и в  period=)
    )
    
    # строим сводную таблицу
    cohort_pivot = cohorts_df.pivot_table(index='cohort', columns='period_number', values='customer_quantity')
    
    # размер когорты = количество пользователей в нулевой период
    cohort_size = cohort_pivot.iloc[:,0]
    
    # строим retention матрицу
    retention_matrix = cohort_pivot.div(cohort_size, axis = 0)

    return retention_matrix


In [ ]:
# создаём retention матрицу используя функцию
retention_matrix = user_retention(purchase_total_df, 'customer_unique_id', 'order_purchase_timestamp')

In [ ]:
# применим форматирование отображения датафрэйма
ur_style = (retention_matrix
            .style 
            .set_caption('Customer retention by cohort')  # добавляем подпись 
            .background_gradient(cmap='viridis')          # раскрашиваем ячейки по столбцам 
            .highlight_null('white')                      # делаем белый фон для значений NaN 
            .format("{:.2%}", na_rep=""))                 # числа форматируем как проценты, NaN заменяем на пустоту 
ur_style

In [ ]:
# когорта с максимальным retention на третий месяц 
# в период с января по декабрь 2017 г.
print(retention_matrix.loc['2017-01':'2017-12', 3].idxmax())


In [ ]:
6. Построить RFM-кластеры для пользователей. Вывести для каждого кластера средние значения метрик R, F, M .
R - recency — давность (как давно ваши клиенты что-то покупали);
F - frequency — частота (как часто клиенты покупают);
M - monetary — деньги (общая сумма покупок).

По каждому из этих признаков мы выделяем по нексолько групп (точное количество определим в процессе). Затем присваиваем каждой группе числовое обозначение от 1 до n, где n - число кластеров

За отчётный период берём один год.

Нам потребуются следующие данные:

id клиента
даты покупок
общее число заказов в течение указанного отчетного периода
средний чек
Взлянем на даты.

In [ ]:
# сегодняшняя дата (действительная)
real_today = (
    pd.to_datetime(
        datetime.datetime.today()
              )
)
real_today

In [ ]:
# дата первого заказа
first_order_date = (
    purchase_total_df
                 .order_purchase_timestamp
                 .min()
)

In [ ]:
# дата последнего заказа
last_order_date = (
    purchase_total_df
                 .order_purchase_timestamp
                 .max()
)

In [ ]:
# разница между сегодняшним днём и датой последнего заказа в месяцах
(real_today.to_period('M') - last_order_date.to_period('M')).n

In [ ]:
# разница между сегодняшним днём и датой первого заказа в месяцах
(real_today.to_period('M') - first_order_date.to_period('M')).n

In [ ]:
# разница между датами последнего и первого заказов в месяцах
(last_order_date.to_period('M') - first_order_date.to_period('M')).n

In [ ]:
Последняя дата заказа - сентябрь 2018 и новее данных нет. Разница между сегодняшней датои и датой последнего заказа почти в два раза больше разницы между первым и последним заказом. Получаестся, что дата первого заказа не сильно отличается от даты последнего заказа по отношению к сегодняшнему дню. С этим связана некоторая сложность в оценке того, что принять за "давнюю дату", а что за "недавнюю".
Чтобы увидеть более реальную картину относительно давности заказов, будем оценивать даты относительно последнего дня заказа, вместо действительного сегодняшнего дня. Т.е. как бы переместимся в прошлое, чтобы посмотреть как вели себя покупатели тогда.

In [ ]:
# условня "сегодняшняя" дата
conditional_today_date = purchase_total_df.order_purchase_timestamp.max()
conditional_today_date


In [ ]:
# дата начала периода
start_date = conditional_today_date - pd.Timedelta(365, unit='day')
start_date

In [ ]:
Подготовим датасет для rfm-анализа. Для этого сгруппируем имеющийся датасет с данными о покупках purchase_total_df и найдём последнюю дату покупок, средний чек и количество заказов на пользователя.
На общую сумму полученную с покупателя может влиять количество покупок и, как следствие, дублирование метрик. Поэтому в качестве параметра Monetary выбрана средняя сумма чека.



In [ ]:
# датафрэём для анализа за весь период времени
pre_rfm_all_years = (
    purchase_total_df
        .copy()

    # сначала сгруппируем по заказам
        .groupby('order_id', as_index=False)                     
        .agg({
            'price': 'sum',                               # найдём сумму заказа
            'customer_unique_id': 'first',                # сохраним колонку пользователя, взяв первый элемент из группы
            'order_purchase_timestamp': 'max'             # оставим последнюю дату
        })
    
    # теперь группируем по пользователям
        .groupby('customer_unique_id', as_index=False)           
        .agg({
            'price': 'mean',                              # средний чек
            'order_id': 'count',                          # количество покупок
            'order_purchase_timestamp': 'max'})           # последнее время заказа
        .rename(columns={
            'price': 'avg_bill', 
            'customer_unique_id': 'customer',
            'order_id': 'purchases_quantity',
            'order_purchase_timestamp': 'last_order'
        })
        .assign(day_quantity = lambda x:                  # добавляем колонку с разницей между условным сегодняшним днём и последней покупкой в днях
                (
                    conditional_today_date.to_period('D') - x.last_order.dt.to_period('D')
                )
                .apply(attrgetter('n'))                   # передадим созданной колонке атрибут n, чтобы задать номинальные единицы изменения (дни)
               )
)


In [ ]:
# датафрэём для анализа за один год
pre_rfm = (
    purchase_total_df
        .copy()
        .query('order_purchase_timestamp >= @start_date') 
    
    # сначала сгруппируем по заказам
        .groupby('order_id', as_index=False)                     
        .agg({
            'price': 'sum',                               # найдём сумму заказа
            'customer_unique_id': 'first',                # сохраним колонку пользователя, взяв первый элемент из группы
            'order_purchase_timestamp': 'max'             # оставим последнюю дату
        })
    
    # теперь группируем по пользователям
        .groupby('customer_unique_id', as_index=False)           
        .agg({
            'price': 'mean',                              # средний чек
            'order_id': 'count',                          # количество покупок
            'order_purchase_timestamp': 'max'})           # последнее время заказа
        .rename(columns={
            'price': 'avg_bill', 
            'customer_unique_id': 'customer',
            'order_id': 'purchases_quantity',
            'order_purchase_timestamp': 'last_order'
        })
        .assign(day_quantity = lambda x:                  # добавляем колонку с разницей между условным сегодняшним днём и последней покупкой в днях
                (
                    conditional_today_date.to_period('D') - x.last_order.dt.to_period('D')
                )
                .apply(attrgetter('n'))                   # передадим созданной колонке атрибут n, чтобы задать номинальные единицы изменения (дни)
               )
)


In [ ]:
pre_rfm.head()

In [ ]:
#Основная сложность проведения анализа — определить границы сегментов. Нам не известно что это за магазин и какими видами товара он торгует. Чтобы понять, что есть норма для этого бизнеса, посмотрим на медианные, средние, минимальные и максимальные значения данных, а также более детально посмотрим на их распределения.

pre_rfm.describe()

In [ ]:
Средний чек: 75-й процентиль 149,9, а максимум - 13440, к тому же среднее и медианное значения отличаются значительно. Среднюю меру здесь описывает медиана 88.34, а высокий максимум обусловлен выбросами - редкими покупками на очень большие суммы.
Количество покупок: подавляющее большинство покупателей делают одну покупку, также как и в случае со средним чеком имеют место выбросы.
Количество дней с последней покупки: здесь картина более-менее равномерная, среднее и медиана почти равны.

In [ ]:
# Monetary
# распределение средних чеков покупателя с логарифмической шкалой Y
sns.displot(pre_rfm.avg_bill, height=7, aspect=2, log_scale=(False, True))
plt.xlabel("Average Bill", size=14)
plt.ylabel("Count (logarithmic scale)", size=14)

In [ ]:
# распределение средних чеков покупателя, левая часть более детально 
sns.displot(pre_rfm.avg_bill[pre_rfm.avg_bill < 500], height=7, aspect=1, kde=True)
plt.xlabel("Average Bill", size=14)
plt.ylabel("Count", size=14)

plt.axvline(x=75,              # синяя пунктирная линия - граница между 1 и 2 группами
            color='blue',
            ls='--', 
            lw=2.5)
plt.axvline(x=280,             # фиолетовая пунктирная линия - граница между 2 и 3 группами
            color='purple',
            ls='--', 
            lw=2.5)

In [ ]:
Для начала попытаемся определит интервалы, интерпретирую графики. Глядя на рапсределение можно выделить следующие группы: От 0 до значения чуть меньшего медианы - 75.
Вторая - свыше 75 до 280.
Третья - свыше 280.
На границах этих групп заметны значительные перепады, или изменение характера апроксимирующей кривой графика.

Далее с попощью метода К-средних определим порги и назначим номера кластеров для каждой из метрик.
Следующим шагом определим оптимальное количство кластеров для колонки со средними чеками с помощью метода локтя. Для этого напишем функцию.

In [ ]:
def optimal_cluster_number(X, n_clucter_max=7):
    '''
    Функция рисует график для определения оптимального количества кластеров, который строится с момощью метода локтя 
    X - пандосовская серия
    n_clucter_max - максимальное количество кластеров
    '''
    error_rates = []
    for i in range(1, n_clucter_max + 1):
        model = KMeans(n_clusters = i, random_state = 42)  # создадим экземпляр класса KMeans
        model.fit(X.values.reshape(-1,1))                                      # преобразуем 1D массив в 2D и обучим модель 
        error_rates.append(model.inertia_)                                     # наполняем список для построения графика

    plt.plot(range(1, n_clucter_max + 1), error_rates)
    plt.xlabel("Number of Cluster", size=14)
    plt.ylabel("Errors", size=14)

In [ ]:
# Находим оптимальное количество кластеров для среднего чека. Сначала попробуем привести колонку со средними чеками к нормальному виду, на сколько это возможно, с помощью Box-Cox трансформации, иначе имеющиеся выбросы исказят результат.

# трнасформируем средний чек и соединим его с нетрансформированным
transformed_data, best_lambda = boxcox(pre_rfm.avg_bill)
trans_avg_bill = pd.DataFrame(transformed_data, columns=['trans_bill'])
trans_avg_bill['avg_bill'] = pre_rfm.avg_bill

In [ ]:
# распределение средних чеков покупателя после Box-Cox трансформации
sns.displot(trans_avg_bill.trans_bill, height=7, aspect=1)
plt.xlabel("Average Bill", size=14)
plt.ylabel("Count", size=14)


In [ ]:
# ищем оптимальное количество кластеров
optimal_cluster_number(trans_avg_bill.trans_bill)

In [ ]:
def get_borders(X, n_clusters):
    '''
    Функция возвращает список границ кластеров
    Принимает на вход:
        X - пандосовская серия
        n_clusters - количество кластеров
    '''
    model = KMeans(n_clusters=n_clusters, random_state = 0)     # создадим экземпляр класса KMeans
    model.fit(X.values.reshape(-1,1))                           # преобразуем 1D массив в 2D и обучим модель 
    clusters = model.labels_                                    # кластеры   

    df = X.to_frame()                                           # переведём серию в датафрэйм
    df['clusters'] = clusters                                   # добавим колонку с кластерами

    borders = [df.iloc[:, 0].min()]                             # добавим минимальное значение в список
    
    for i in range(n_clusters):
        border = df.query('clusters == @i').iloc[:, 0].max()    # отбираем максимальные значения при определённых значениях clusters
        borders.append(border)                                  # добавляем в список
    
    return sorted(borders)                                      # возвращаем сортированный список границ

In [ ]:
# границы трансформированного прайса
m_borders_trans = get_borders(trans_avg_bill.trans_bill, 3)
m_borders_trans

In [ ]:
# переведём границы трансформированного прайса в нетрансформированные
m_borders = []
for i in m_borders_trans:
    m_borders.append(trans_avg_bill[trans_avg_bill.trans_bill == i].avg_bill.values[0])

In [ ]:
# границы интервалов Monetary
m_borders

In [ ]:
# распределение средних чеков покупателя, с нанесёнными границами (левая часть более детально)
sns.displot(pre_rfm.avg_bill[pre_rfm.avg_bill < 500], height=7, aspect=1, kde=True)
plt.xlabel("Average Bill", size=14)
plt.ylabel("Count", size=14)

plt.axvline(x=m_borders[1],              # синяя пунктирная линия - граница между 1 и 2 группами
            color='blue',
            ls='--', 
            lw=2.5)
plt.axvline(x=m_borders[2],             # фиолетовая пунктирная линия - граница между 2 и 3 группами
            color='purple',
            ls='--', 
            lw=2.5)

In [ ]:
# назначаем рейтинг для Monetary
m_labels = (1, 2, 3)
m_score_bins = m_borders

m_score = (
    pd.cut(pre_rfm.avg_bill, 
                   bins=m_score_bins, 
                   labels = m_labels, 
                   right=True,
                   include_lowest = True)
)

In [ ]:
Frequency

In [ ]:
# доля пользователей, сделавших только один заказ
single_order_perc = (
    pre_rfm.purchases_quantity.value_counts()[1]
    /
    pre_rfm.purchases_quantity.count()
)
print(f'{100 * single_order_perc:.2f}% пользователей сделали только один заказ.')

In [ ]:
# распределение количества заказов покупателя с логарифмической шкалой Y
sns.displot(pre_rfm.purchases_quantity, height=7, aspect=1, log_scale=(False, True))
plt.xlabel("Number Of Orders per Customer", size=14)
plt.ylabel("Count (logarithmic scale)", size=14)

In [ ]:
# распределение количества заказов на покупателя с количеством покупок <= 3
sns.displot(pre_rfm.purchases_quantity[pre_rfm.purchases_quantity <= 3], height=7)
plt.xlabel("Number Of Orders per Customer", size=14)
plt.ylabel("Count", size=14)


In [ ]:
# распределение количества заказов на покупателя с количеством покупок >= 3
sns.displot(pre_rfm.purchases_quantity[pre_rfm.purchases_quantity >= 3], height=7)
plt.xlabel("Number Of Orders per Customer", size=14)
plt.ylabel("Count", size=14)


In [ ]:
Аналогичным образом, попытаемся определит интервалы интерпретируюя графики.
Глядя на цифры и распределения видим, что общую массу покупателей можно разделить на 3 группы по характерным "ступеням" - резким перепадам, показывающим своеобразный переход на новый уровень лояльности к магазину.
Преобладают покупатели с одним количеством заказов - 71240 (97.49%). Эту часть покупателей определим в группу №1. Число покупателей, сделавших только 2 покупки - 1709 - группа №2. В третью группу возьмём оставшихся с числом покупок 3 и более.
Итак:
Группа 1 - только один заказ.
Группа 2 - только два заказа.
Группа 3 - 3 и более заказов.

Далее воспользуемся кластеризацией методом k-средних и уточним выбранные границы.
Находим оптимальное количество кластеров для количества покупок с помощью k-means и метода локтя. В данносм случае не требуетются какие-либо преобразования распределения.



In [ ]:
# оптимальное количество кластеров для purchases_quantity
optimal_cluster_number(pre_rfm.purchases_quantity)

In [ ]:
# интервалы для Frequency
f_borders = get_borders(pre_rfm
                            .purchases_quantity
                        , 3)
f_borders[0] = 0                                # присвоим первому (нулевому) элементу значение 0, вместо 1, чтобы cut() мог обработать этот список
f_borders

In [ ]:
# назначаем рейтинг для Frequency
f_labels = (1, 2, 3)
f_score_bins = f_borders

f_score = (
    pd.cut(pre_rfm.purchases_quantity, 
                   bins=f_score_bins, 
                   labels = f_labels, 
                   right=True, 
                   include_lowest=True)
          )


In [ ]:
# Recency
# распределение колонки давности последней покупки
sns.displot(pre_rfm.day_quantity, height=7, aspect=1, kde=True)
plt.xlabel("Days Since Last Purchase per Customer", size=14)
plt.ylabel("Count", size=14)

plt.axvline(x=pre_rfm.day_quantity.median(),               # зелёная пунктирная линия - медиана
            color='green',
            ls='--', 
            lw=2.5)
plt.axvline(x=pre_rfm.day_quantity.quantile(q=0.33),       # синяя пунктирная линия - 33-й процентиль
            color='blue',
            ls='--', 
            lw=2.5)
plt.axvline(x=pre_rfm.day_quantity.quantile(q=0.66),       # фиолетовая пунктирная линия - 66-й процентиль
            color='purple',
            ls='--', 
            lw=2.5)

In [ ]:
# 33-й и 66-й процентили количества дней от последней покупки
pre_rfm.day_quantity.quantile(q=[0.33, 0.66])

In [ ]:
# оптимальное количество кластеров для day_quantity
optimal_cluster_number(pre_rfm.day_quantity)

In [ ]:
# границы разбивки для recency
r_borders = get_borders(pre_rfm.day_quantity, 3)
r_borders

In [ ]:
# назначаем рейтинг для recency
r_labels = (3, 2, 1)
r_score_bins = r_borders

r_score = (pd.cut(pre_rfm.day_quantity, 
                   bins=r_score_bins, 
                   labels = r_labels, 
                   right=True, 
                   include_lowest=True)
          )

In [ ]:
# создадим общий датафрэйм, добавив в него рейтинги R, F, M
rfm_kmeans = (
    pre_rfm
        .copy()
        .drop('last_order', axis=1)
        .assign(recency   = r_score,
                frequency = f_score,
                monetary  = m_score)
)

In [ ]:
rfm_kmeans.head()


In [ ]:
# среднее recency по кластерам для k-means
avg_recency_kmeans = (
    rfm_kmeans
        .pivot_table(index='recency', values='day_quantity')
        .reset_index()
        .rename(columns={'recency': 'cluster',
                        'day_quantity': 'recency'})
)

In [ ]:
# среднее frequency по кластерам для k-means
avg_frequency_kmeans = (
    rfm_kmeans
        .pivot_table(index='frequency', values='purchases_quantity')
        .reset_index()
        .rename(columns={'frequency': 'cluster',
                        'purchases_quantity': 'frequency'})
)

In [ ]:
# среднее monetary по кластерам для k-means
avg_monetary_kmeans = (
    rfm_kmeans
        .pivot_table(index='monetary', values='avg_bill')
        .reset_index()
        .rename(columns={'monetary': 'cluster',
                        'avg_bill': 'monetary'})
)

In [ ]:
# сводная таблица средних значений recency, frequency и monetary по кластерам
avg_rfm_kmeans = (
    avg_recency_kmeans
        .merge(avg_frequency_kmeans, on='cluster')
        .merge(avg_monetary_kmeans, on='cluster')
        .sort_values('cluster')
        .reset_index(drop=True)
)

In [ ]:
Средние значения метрик R, F, M для каждого кластера

In [ ]:
avg_rfm_kmeans

In [ ]:
# добавляем колонку RFM
rfm_kmeans['RFM'] = rfm_kmeans.apply(lambda x: str(x.recency) + str(x.frequency) + str(x.monetary), axis=1)

In [ ]:
rfm_kmeans.head()

In [ ]:
# количество пользователей по каждому сегменту
customer_count = (
            rfm_kmeans
                .groupby('RFM', as_index=False)
                .agg({'customer': 'count'})
                .rename(columns={'customer': 'customer_number'})
)

In [ ]:
#Следующая таблица отражает сегменты и средние значения recency, frequency и monetary для них, а также количество пользователей по каждому сегменту.

rfm_mean = (
    rfm_kmeans
        .groupby('RFM', as_index=False)
        .agg({'day_quantity':      'mean',
             'purchases_quantity': 'mean',
             'avg_bill':           'mean'})
        
        # переименуем колонки
        .rename(columns={'day_quantity':      'recency_mean',
                        'purchases_quantity': 'frequency_mean',
                        'avg_bill':           'monetary_mean'})

        # добавим количество пользователей по сегментам
        .merge(customer_count, on='RFM')
)

In [ ]:
rfm_mean